In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pickle
import random
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor(),  
])
class SiameseSignatureDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        img1, img2, label = self.pairs[idx]
        if not torch.is_tensor(img1):
            img1 = transform(img1)
        if not torch.is_tensor(img2):
            img2 = transform(img2)
        label = torch.tensor(label, dtype=torch.float32)
        return img1, img2, label
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        resnet = models.resnet18(pretrained=False)
        resnet.fc = nn.Identity()
        self.base_model = resnet  
        self.embedding = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128)
        )
    def forward_once(self, x):
        x = self.base_model(x)
        x = self.embedding(x)
        return x
    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)
with open("C:/Active-Repositories/secure-signature-verification/image_pairs/positive_pairs.pkl", "rb") as f1:
    positive_pairs = pickle.load(f1)
with open("C:/Active-Repositories/secure-signature-verification/image_pairs/negative_pairs.pkl", "rb") as f2:
    negative_pairs = pickle.load(f2)
all_pairs = positive_pairs + negative_pairs
random.shuffle(all_pairs)
split = int(0.8 * len(all_pairs))
train_pairs = all_pairs[:split]
test_pairs = all_pairs[split:]
test_dataset = SiameseSignatureDataset(test_pairs)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
parameters_file_path = "C:/Active-Repositories/secure-signature-verification/trained_model_parameters/siamese_resnet_18.pth"
model.load_state_dict(torch.load(parameters_file_path, map_location=device))
model.eval()
correct = 0
total = 0
threshold = 0.4275 
with torch.no_grad():
    for img1, img2, labels in test_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        out1, out2 = model(img1, img2)
        distances = F.pairwise_distance(out1, out2)
        predictions = (distances < threshold).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


c:\Active-Repositories\secure-signature-verification\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Active-Repositories\secure-signature-verification\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Accuracy: 99.98%
